In [1]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 1. Load documents
loader = PyPDFDirectoryLoader(r"C:\Users\zaid ahmed\OneDrive\Desktop\data")
the_text = loader.load()
print(f"Loaded {len(the_text)} documents.")

# 2. Split text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(the_text)
print(f"Split into {len(docs)} chunks.")

# 3. Verify chunks are non-empty
if not docs or not docs[0].page_content.strip():
    raise ValueError("No valid text chunks found after splitting!")

# 4. Test embeddings
embeddings = OllamaEmbeddings(model="nomic-embed-text")
try:
    test_embedding = embeddings.embed_query("Test")
    print("Embedding model working. Vector length:", len(test_embedding))
except Exception as e:
    raise RuntimeError(f"Embedding model failed: {e}")

# 5. Create vector store
vectorstore = Chroma.from_documents(
    documents=docs,
    collection_name="ollama_embeds",
    embedding=embeddings,
)
print("Vector store created!")
retriever = vectorstore.as_retriever()

Loaded 19 documents.
Split into 71 chunks.


C:\Users\zaid ahmed\AppData\Local\Temp\ipykernel_10628\679499834.py:21: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="nomic-embed-text")


Embedding model working. Vector length: 768
Vector store created!


In [2]:
from langchain_groq import ChatGroq
from env import groq_api_key
llm = ChatGroq(
            groq_api_key=groq_api_key,
            model_name='llama-3.3-70b-versatile'
    )


#https://github.com/ollama/ollama mixtral-8x7b-32768

# from langchain_community.llms import Ollama

# llm = Ollama(model="llama2")

# llm.invoke("Tell me a joke")

In [12]:
rag_template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
rag_prompt = ChatPromptTemplate.from_template(rag_template)
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [13]:
import os
import time
import textwrap
import gradio as gr

# Test the architecture with a simple hard coded question
response = rag_chain.invoke("What is this document about")
print(textwrap.fill(response, width=80))

This document appears to be a transcript of a conference call or earnings call
for SJS Enterprises, a company that has recently made an acquisition of Walter
Pack. The call discusses the company's financial performance, guidance, and
strategy, including the impact of the acquisition on their earnings and margins.
The participants also discuss the company's product lines, including consumer
durables and exports, and answer questions from analysts.


In [14]:
# Make the questions dynamic using a chat interface. Let's use gradio for this.
def process_question(user_question):
    start_time = time.time()

    # Directly using the user's question as input for rag_chain.invoke
    response = rag_chain.invoke(user_question)

    # Measure the response time
    end_time = time.time()
    response_time = f"Response time: {end_time - start_time:.2f} seconds."

    # Combine the response and the response time into a single string
    full_response = f"{response}\n\n{response_time}"

    return full_response

# Setup the Gradio interface
iface = gr.Interface(fn=process_question,
                     inputs=gr.Textbox(lines=2, placeholder="Type your question here..."),
                     outputs=gr.Textbox(),
                     title="Personal Knowledge Chat App",
                     description="Ask any question about your document, and get an answer along with the response time.")

# Launch the interface
iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://f3f5abe22535037465.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
